<a href="https://colab.research.google.com/github/Recollectionss/University/blob/main/lab_2/lab_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np

BORDER_LENGTH = 100

def print_border():
    print("-" * BORDER_LENGTH)

def create_matrix_and_vector(n):
    A = np.zeros((n, n))
    # b = np.random.randint(1, 10, size=n)
    b = np.array([7, 1, 9, 4])

    for i in range(n):
        A[i][i] = 10  # Діагональні елементи для домінантності

        if i > 0:
            A[i][i-1] = 2  # Піддіагональні елементи
        if i < n - 1:
            A[i][i+1] = 2  # Наддіагональні елементи

    return A, b

def check_gauss(matr):
    print("\nChecking if the system can be solved by Gauss method:")
    det = np.linalg.det(matr)
    print(f"Determinant: {det:.4f}")

    if abs(det) < 1e-10:
        print("Determinant is too close to 0 -> cannot be solved")
        print_border()
        return False

    print("Determinant is non-zero, solving can proceed.")
    print_border()
    return True

def gauss(A, b):
    print("=== GAUSS METHOD ===\n")

    if not check_gauss(A):
        return np.zeros(len(b))

    matr_len = len(b)
    for i in range(matr_len):
        p = np.identity(matr_len)
        m = np.identity(matr_len)

        print(f"\nStep {i + 1}:")
        print("Current A matrix:\n", A)
        print("\nCurrent b vector:\n", b)

        max_row = np.argmax(np.abs(A[i:matr_len, i])) + i
        if np.abs(A[max_row, i]) < 1e-10:  # Додано перевірку на нульовий опорний елемент
            print("Pivot element is too small, system may be singular")
            return np.zeros(len(b))

        p[[i, max_row]] = p[[max_row, i]]
        print("\nP matrix (permutation):\n", p)

        A = np.matmul(p, A)
        b = np.matmul(p, b)

        for j in range(matr_len):
            if j < i:
                m[j, i] = 0
            elif j == i:
                m[j, i] = 1 / A[j, i]
            else:
                m[j, i] = -A[j, i] / A[i, i]

        print("\nM matrix (elimination):\n", m)
        print_border()

        A = np.matmul(m, A)
        b = np.matmul(m, b)

    print("\nFinal transformed A matrix:\n", A)
    print("\nFinal transformed b vector:\n", b)
    print_border()

    x = np.zeros(matr_len)
    for i in range(matr_len - 1, -1, -1):
        x[i] = b[i]
        for j in range(i + 1, matr_len):
            x[i] -= A[i, j] * x[j]

    return x

def gauss_extended(A, b):
    print("=== GAUSS METHOD (Extended) ===\n")

    augmented_matrix = np.hstack((A, b.reshape(-1, 1)))

    for i in range(len(A)):
        max_row = np.argmax(np.abs(augmented_matrix[i:, i])) + i
        if np.abs(augmented_matrix[max_row, i]) < 1e-10:
            print("Pivot element is too small, system may be singular")
            return np.zeros(len(b))

        augmented_matrix[[i, max_row]] = augmented_matrix[[max_row, i]]

        for j in range(i + 1, len(A)):
            ratio = augmented_matrix[j, i] / augmented_matrix[i, i]
            augmented_matrix[j] -= ratio * augmented_matrix[i]

    x = np.zeros(len(b))
    for i in range(len(A) - 1, -1, -1):
        x[i] = (augmented_matrix[i, -1] - np.dot(augmented_matrix[i, i+1:len(A)], x[i+1:])) / augmented_matrix[i, i]

    return x

def gauss_determinant(A):
    det = 1
    matr_len = len(A)
    for i in range(matr_len):
        max_row = np.argmax(np.abs(A[i:, i])) + i
        if A[max_row, i] == 0:
            print("Matrix is singular, determinant is 0.")
            return 0
        if max_row != i:
            A[[i, max_row]] = A[[max_row, i]]
            det *= -1  # Зміна знаку визначника при перестановці рядків
        det *= A[i, i]
        for j in range(i + 1, matr_len):
            ratio = A[j, i] / A[i, i]
            A[j] -= ratio * A[i]
    return det

def print_tridiagonal_step(step, A, b):
    print(f"Step {step + 1}:")
    print("Matrix A:\n", np.round(A, 5))
    print("Vector b:\n", np.round(b, 5))
    print_border()

def tridiagonal_solver(A, b):
    n = len(b)
    c = np.zeros(n - 1)
    d = np.zeros(n)

    # Пряма прогонка
    for i in range(1, n):
        factor = A[i, i - 1] / A[i - 1, i - 1]
        A[i, i] -= factor * A[i - 1, i]
        b[i] -= factor * b[i - 1]
        print_tridiagonal_step(i - 1, A, b)

    # Зворотна прогонка
    d[-1] = b[-1] / A[-1, -1]
    for i in range(n - 2, -1, -1):
        d[i] = (b[i] - A[i, i + 1] * d[i + 1]) / A[i, i]
        print(f"Backward Step {n - i - 1}: d = {np.round(d, 5)}")
        print_border()

    return d

def is_diagonally_dominant(A):
    for i in range(len(A)):
        row_sum = np.sum(np.abs(A[i])) - np.abs(A[i, i])
        if np.abs(A[i, i]) <= row_sum:
            return False
    return True

def print_iteration_values(iteration, x):
    print(f"Iteration {iteration + 1}: x = {np.round(x, 5)}")

def jacobi_method(A, b, epsilon=1e-6, max_iterations=1000):
    n = len(A)
    x = np.zeros(n)

    # Обчислення коефіцієнта q (норма матриці ітерацій)
    D = np.diag(np.diag(A))  # Діагональна частина A
    R = A - D  # Решта елементів матриці A
    T = np.linalg.inv(D) @ R  # Ітераційна матриця
    q = np.linalg.norm(T, 1)  # Норма 1 для ітераційної матриці

    for iteration in range(1, max_iterations + 1):
        x_new = np.zeros(n)
        for i in range(n):
            s = sum(A[i, j] * x[j] for j in range(n) if j != i)
            x_new[i] = (b[i] - s) / A[i, i]

        print_iteration_values(iteration - 1, x_new)

        # Перевірка критерію зупинки
        if (q**iteration) / (1 - q) <= epsilon:
            print(f"Converged after {iteration} iterations based on the q criterion.")
            return x_new

        x = x_new

    print("Did not converge within the maximum number of iterations.")
    return x

def inverse_matrix(A):
    n = A.shape[0]
    I = np.eye(n)  # Одинична матриця
    augmented_matrix = np.hstack((A, I))  # Розширена матриця [A | I]

    print("Initial augmented matrix [A | I]:\n", augmented_matrix, "\n")

    # Прямий хід
    for i in range(n):
        max_row = np.argmax(np.abs(augmented_matrix[i:, i])) + i
        if augmented_matrix[max_row, i] == 0:
            raise np.linalg.LinAlgError("Matrix is singular and cannot be inverted.")

        # Перестановка рядків
        if max_row != i:
            augmented_matrix[[i, max_row]] = augmented_matrix[[max_row, i]]

        # Зведення до одиничного елемента на діагоналі
        augmented_matrix[i] /= augmented_matrix[i, i]

        print(f"Forward Elimination - Step {i + 1}:\n", augmented_matrix, "\n")

        # Обнулення елементів нижче
        for j in range(i + 1, n):
            ratio = augmented_matrix[j, i]
            augmented_matrix[j] -= ratio * augmented_matrix[i]

    # Зворотний хід
    for i in range(n - 1, -1, -1):
        for j in range(i - 1, -1, -1):
            ratio = augmented_matrix[j, i]
            augmented_matrix[j] -= ratio * augmented_matrix[i]

        print(f"Back Substitution - Step {n - i}:\n", augmented_matrix, "\n")

    return augmented_matrix[:, n:]  # Повертаємо обернену матрицю

def main():
    A, b = create_matrix_and_vector(4)

    print("=== GENERATED MATRIX A ===\n", A)
    print("=== GENERATED VECTOR b ===\n", b)

    # Визначник матриці
    A_copy = A.copy()
    determinant = gauss_determinant(A_copy)
    print("\nDeterminant of matrix A by Gauss method:", determinant)
    print_border()

    # Знаходимо обернену матрицю
    try:
        A_inv = inverse_matrix(A.copy())
        print("\nInverse of matrix A:\n", A_inv)
    except np.linalg.LinAlgError:
        print("Matrix A is singular and cannot be inverted.")

    # Метод Гаусса для звичайної матриці
    solution_gauss = gauss(A.copy(), b.copy())
    print("\nSolution using Gauss method:\n", solution_gauss)

    if determinant != 0:
        if is_diagonally_dominant(A):
            try:
                epsilon_input = input("Enter precision for Jacobi method (press Enter to use default 1e-6): ")
                epsilon = float(epsilon_input) if epsilon_input else 1e-6
                solution_jacobi = jacobi_method(A.copy(), b.copy(), epsilon=epsilon)
                print("\nSolution using Jacobi method:\n", solution_jacobi)
            except ValueError:
                print("Invalid input. Using default precision 1e-6.")
                solution_jacobi = jacobi_method(A.copy(), b.copy())
                print("\nSolution using Jacobi method:\n", solution_jacobi)
        else:
            print("Matrix is not diagonally dominant. Jacobi method convergence is not guaranteed.")

    # Метод прогонки
    solution_tridiagonal = tridiagonal_solver(A.copy(), b.copy())
    print("\nSolution using Tridiagonal method:\n", solution_tridiagonal)


if __name__ == "__main__":
    main()


=== GENERATED MATRIX A ===
 [[10.  2.  0.  0.]
 [ 2. 10.  2.  0.]
 [ 0.  2. 10.  2.]
 [ 0.  0.  2. 10.]]
=== GENERATED VECTOR b ===
 [7 1 9 4]

Determinant of matrix A by Gauss method: 8816.0
----------------------------------------------------------------------------------------------------
Initial augmented matrix [A | I]:
 [[10.  2.  0.  0.  1.  0.  0.  0.]
 [ 2. 10.  2.  0.  0.  1.  0.  0.]
 [ 0.  2. 10.  2.  0.  0.  1.  0.]
 [ 0.  0.  2. 10.  0.  0.  0.  1.]] 

Forward Elimination - Step 1:
 [[ 1.   0.2  0.   0.   0.1  0.   0.   0. ]
 [ 2.  10.   2.   0.   0.   1.   0.   0. ]
 [ 0.   2.  10.   2.   0.   0.   1.   0. ]
 [ 0.   0.   2.  10.   0.   0.   0.   1. ]] 

Forward Elimination - Step 2:
 [[ 1.          0.2         0.          0.          0.1         0.
   0.          0.        ]
 [ 0.          1.          0.20833333  0.         -0.02083333  0.10416667
   0.          0.        ]
 [ 0.          2.         10.          2.          0.          0.
   1.          0.        ]
 [ 0.